In [ ]:
from cloudyfsps import outObj as ob
from cloudyfsps.outObj import getColors, nColors, add_dopita
from cloudyfsps.generalTools import calcQ
import numpy as np
import matplotlib.pyplot as plt
from vz import plot_bpt as vzBPT
import matplotlib as mpl
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rc('font', size=16, family='serif', serif=[r'cmr10'], style='normal', variant='normal', stretch='normal', weight='heavy')
mpl.rc('legend', labelspacing=0.1, handlelength=2, fontsize=12)
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputCKC/'
mod_prefix='ZAU'
zau = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputMIST/'
mod_prefix='ZAU'
mau = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputCSFH/'
mod_prefix='ZAU'
csf = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
import pickle
from numpy.lib import recfunctions

samplefile = open('../SNcuts_kappa20_sample.pickle', 'rb')
sample = pickle.load(samplefile)
samplefile.close()
# add fields to recarr
sample = recfunctions.append_fields(sample, ['log_OIII_Hb','log_SII_Ha'],
                                    [np.log10(sample['OIII5007']/sample['Hbeta']),
                                    np.log10((sample['SII6717']+sample['SII6731'])/sample['Halpha'])],
                                    asrecarray=True)

In [ ]:
from matplotlib.colors import LogNorm
def get_contours(x, y, bins=(50, 50), ranges=None):
    if ranges:
        H, xedges, yedges = np.histogram2d(x, y, bins=bins, range=ranges)
    else:
        H, xedges, yedges = np.histogram2d(x, y, bins=bins)
    xmid = (xedges[1:] + xedges[:-1]) / 2.0
    ymid = (yedges[1:] + yedges[:-1]) / 2.0
    return xmid, ymid, H.T
def plot_2Dhist(x, y, xlabel=None, ylabel=None, cblabel=None, ranges=[[-0.007, 0.002],[-0.014, 0.005]],
                vmin=1, vmax=10**5, normed=False, plot_cb=True,
                filename=None, annotate_string=None, annotate_loc=None, ax=None, cmap=plt.cm.viridis):
    xmid, ymid, H = get_contours(x, y, bins=(50, 50), ranges=ranges)
    
    if normed == True:
        norm = np.sum(H, axis=0)  # get total number of galaxies in each mass bin
        # hacky way to make sure division is done correctly
        H_norm = np.zeros(np.shape(H))
        for col in range(np.shape(H)[1]):
            H_norm[:, col] = H[:, col] / norm[col]
    else:
        H_norm = H
    if ax is None:
        fig, ax = plt.subplots(figsize=(6.5, 5))
    #plt.gcf().subplots_adjust(bottom=0.15)
    # LogNorm was 0.001, 0.5
    ax.imshow(H_norm, origin='lower', aspect='auto',
              interpolation='nearest', cmap=cmap, norm=LogNorm(vmin=vmin, vmax=vmax),
              extent=(ranges[0][0], ranges[0][1], ranges[1][0], ranges[1][1]), vmin=vmin,
              vmax=vmax)  # cmap=plt.cm.binary, cmap=plt.cm.cubehelix_r

    if xlabel:
        plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)
    if plot_cb:
        cb = plt.colorbar()
    if cblabel:
        cb.set_label(cblabel)
    
    if annotate_string:
        plt.annotate(annotate_string, annotate_loc, fontsize=16)
    return

In [ ]:
def altBPT(zau, age=2.82e9, ax=None, xname='log_NIIb_Ha', yname='log_OIIIb_Hb', do_label=False,
           grid_xname='logU', grid_yname='logZ', xlim=(-2.0, 1.0), ylim=(-2.0, 1.0),
           xcname='Blues', ycname='inferno', lw=4, add_starlab=True,alpha=1.0,
           xclims=(-4.0, -1.0), yclims=(-2.0, 0.5), xv=(0.2, 1.0), yv=(0.2, 0.9), add_star=True):
    if ax is None:
        fig, ax = plt.subplots(1)
    use_mods = [mod for mod in zau.mods if ((mod.age == age) and \
                                            (mod.__getattribute__(grid_xname) >= xclims[0]) and \
                                            (mod.__getattribute__(grid_xname) <= xclims[1]) and \
                                            (mod.__getattribute__(grid_yname) >= yclims[0]) and \
                                            (mod.__getattribute__(grid_yname) <= yclims[1]))]
    grid_x = np.array([x for x in zau.__getattribute__(grid_xname+'_vals') if (x >= xclims[0]) & (x <= xclims[1])])
    grid_y = np.array([y for y in zau.__getattribute__(grid_yname+'_vals') if (y >= yclims[0]) & (y <= yclims[1])])
    
    sMx = getColors(grid_x, cname=xcname, maxv=xv[1], minv=xv[0])
    sMy = getColors(grid_y, cname=ycname, maxv=yv[1], minv=yv[0])
    FLxx = []
    FLxy = []
    for gx in grid_x:
        mods = [mod for mod in use_mods if (mod.__getattribute__(grid_xname) == gx)]
        x_x = np.array([mod.__getattribute__(xname) for mod in mods])
        x_y = np.array([mod.__getattribute__(yname) for mod in mods])
        ax.plot(x_x, x_y, lw=lw, color=sMx.to_rgba(gx), markeredgecolor='None', alpha=alpha)
        if gx == grid_x[0] or gx == grid_x[-1]:
            FLxx.append(x_x)
            FLxy.append(x_y)
    FLyx = []
    FLyy = []
    for gy in grid_y:
        mods = [mod for mod in use_mods if (mod.__getattribute__(grid_yname) == gy)]
        y_x = np.array([mod.__getattribute__(xname) for mod in mods])
        y_y = np.array([mod.__getattribute__(yname) for mod in mods])
        ax.plot(y_x, y_y, lw=lw, color=sMy.to_rgba(gy), markeredgecolor='None', alpha=alpha)
        if gy == grid_y[0] or gy == grid_y[-1]:
            FLyx.append(y_x)
            FLyy.append(y_y)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    if add_star:
        x1,y1 = FLxx[1][0], FLxy[1][0]
        ax.plot(x1, y1, '*', markersize=18,
                color=sMy.to_rgba(grid_y[0]),
                markeredgecolor='None', zorder=15)
        if add_starlab:
            ax.annotate(r'$\log\;$Z$/$Z$_{{\odot}}={0:.1f}$'.format(grid_y[0]),
                        xy=(x1,y1), xycoords='data',
                        xytext=(15,10), textcoords='offset points',
                        ha='left', va='top', size=14)
        x2,y2=FLyx[1][0], FLyy[1][0]
        ax.plot(x2, y2, '*', markersize=22, color=sMy.to_rgba(grid_y[-1]),
                markeredgecolor='None', zorder=15)
        if add_starlab:
            ax.annotate(r'$\log\;$Z$/$Z$_{{\odot}}={0:.1f}$'.format(grid_y[-1]),
                        xy=(x2, y2), xycoords='data',
                        xytext=(-10,0), textcoords='offset points',
                        ha='right', va='top', size=14)
    return

# Figure 22

# Figure 23

In [ ]:
sxname = 'logNHa'
syname = 'log_OIII_Hb'
xlim=[-1.7, 0.1]
ylim=[-2.4, 1.2]

ratio='NIIb'
age = 1.0e6
xlab=r'$\log$ [N II]$\;\lambda$6584 / H$\alpha$'
ylab=r'$\log$ [O III]$\;\lambda$5007 / H$\beta$'
xname, yname = 'log_NIIb_Ha', 'log_OIIIb_Hb'

ages = [1.0e6, 2.0e6, 3.0e6, 4.0e6, 5.0e6, 6.0e6]
fig, axes = plt.subplots(2,3, figsize=(9, 6))
for age, ax in zip(ages, fig.axes):
    add_starlab=False
    if age == 5.0e6:
        add_starlab=True
    plot_2Dhist(sample[sxname], sample[syname], ranges=[[-2.0,0.0], ylim], ax=ax,
                cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
    #vzBPT(True, ax=ax, line_ratio=ratio)
    altBPT(zau, age=age, ax=ax, xname=xname, yname=yname, alpha=0.7,add_star=False,
           yclims=(-1.0, 0.3), ycname='Blues', add_starlab=add_starlab,
           xlim=xlim, ylim=ylim,xv=(0.8, 0.8), yv=(0.8, 0.8), lw=2)
    lab = "{0:.0f} Myr".format(age/1.e6)
    ax.annotate(lab, xy=(0.95, 0.95), xycoords='axes fraction', ha='right', va='top', size=20)
    ax.set_xlabel(' ')
    ax.set_ylabel(' ')
    NII = np.linspace(-4.5, 0.04)
    OIII = 0.61/(NII - 0.05) + 1.3
    ax.plot(NII, OIII, '--', color='k', lw=2)
    NII = np.linspace(-4.5, 0.46, 100)
    OIII = 0.61/(NII - 0.47) + 1.19
    ax.plot(NII, OIII, color='k', lw=2)
#fig.axes[0].annotate("Padova+Geneva", xy=(0.02, 0.02),
#                     xycoords='axes fraction', ha='left', va='bottom', size=24)
for age, ax in zip(ages, fig.axes):
    add_starlab=False
    if age == 5.0e6:
        add_starlab=True
    #plot_2Dhist(sample[sxname], sample[syname], ranges=[[-2.0,0.0], ylim], ax=ax,
    #            cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
    #vzBPT(True, ax=ax, line_ratio=ratio)
    altBPT(mau, age=age, ax=ax, xname=xname, yname=yname, alpha=0.7,add_star=False,
           yclims=(-1.0, 0.3), ycname='Reds', xcname='Reds', add_starlab=add_starlab,
           xlim=xlim, ylim=ylim,xv=(0.8, 0.8), yv=(0.8, 0.8), lw=2)
    lab = "{0:.0f} Myr".format(age/1.e6)
    ax.annotate(lab, xy=(0.95, 0.95), xycoords='axes fraction', ha='right', va='top', size=20)
    ax.set_xlabel(' ')
    ax.set_ylabel(' ')
#fig.axes[3].annotate("MIST", xy=(0.02, 0.02), xycoords='axes fraction',
#                     ha='left', va='bottom', size=24)
plt.setp([a.get_xticklabels() for a in fig.axes[0:2]], visible=False);

plt.setp([a.get_yticklabels() for a in fig.axes], visible=False);
plt.setp(fig.axes[0].get_yticklabels(), visible=True);
plt.setp(fig.axes[3].get_yticklabels(), visible=True);
fig.subplots_adjust(left=0.08, wspace=0, hspace=0, right=0.98, top=0.98, bottom=0.1)
#fig.axes[2].set_xlabel(xlab, size=14)
fig.axes[3].set_xlabel(xlab, size=14)
fig.axes[4].set_xlabel(xlab, size=14)
fig.axes[5].set_xlabel(xlab, size=14)
fig.axes[0].set_ylabel(ylab, size=14)
fig.axes[3].set_ylabel(ylab, size=14)

fig.axes[-1].annotate(r'Padova+Geneva', xy=(0.02, 0.3), xycoords='axes fraction', fontsize=20,
                     xytext=(0.2,0.7), textcoords='axes fraction', ha='left', va='top',
                     arrowprops=dict(arrowstyle='wedge', connectionstyle='arc3, rad=0.68', fc='dodgerblue',alpha=0.9, ec='None'))
fig.axes[-1].annotate(r'MIST', xy=(0.02, 0.4), xycoords='axes fraction', fontsize=20,
                     xytext=(0.2,0.8), textcoords='axes fraction', ha='left', va='top',
                     arrowprops=dict(arrowstyle='wedge', connectionstyle='arc3, rad=0.71', fc='Red', alpha=0.7, ec='None'))

fig.savefig('./f23.pdf', dpi=300)

# Figure 24

In [ ]:
ages = mau.age_vals[2:8]

ucolors = nColors(len(mau.logU_vals), minv=0.2, maxv=1.0, cname='Blues')
zcolors = nColors(len(mau.logZ_vals), minv=0.1, maxv=0.85, cname='CMRmap')
fig, axes = plt.subplots(2,3, figsize=(10,6))
for age, ax in zip(ages, fig.axes):
    X,Y,Zx = zau.group_mods(zval='log_OIII_OII', xlims=(-2.6, 0.6),
                            yval='logU',
                            const='age', cval=age)
    X,Y,Zy = zau.group_mods(zval='log_HeII_Hb', xlims=(-2.6, 0.6),
                            yval='logU', const='age', cval=age)
    for i in range(X.shape[0]):
        ax.plot(Zx[i,:], Zy[i,:], lw=3, color='grey', alpha=0.5) # lines of constant U
    for i in range(X.shape[1]):
        ax.plot(Zx[:,i], Zy[:,i], lw=3, color='grey', alpha=0.5)

for age, ax in zip(ages, fig.axes):
    X,Y,Zx = mau.group_mods(zval='log_OIII_OII', xlims=(-2.6, 0.6),
                            yval='logU',
                            const='age', cval=age)
    X,Y,Zy = mau.group_mods(zval='log_HeII_Hb', xlims=(-2.6, 0.6),
                            yval='logU', const='age', cval=age)
    for i in range(X.shape[0]):
        ax.plot(Zx[i,:], Zy[i,:], lw=3, color=ucolors[i]) # lines of constant U
    for i in range(X.shape[1]):
        ax.plot(Zx[:,i], Zy[:,i], lw=3, color=zcolors[i])
    #print X[-1,0], Y[-1,0]
    ax.plot(Zx[-1,0], Zy[-1,0], '*', markersize=18, color='k')
    if age == ages[0]:
        ax.annotate(r'log Z/Z$_{\odot}$=-2', size=14,
                    xy=(Zx[-1,0], Zy[-1,0]), xycoords='data',
                    xytext=(0,5), textcoords='offset points',
                    ha='right', va='bottom')
        ax.annotate(r'$\log\mathcal{U}_0$=-1', size=14,
                    xy=(Zx[-1,0], Zy[-1,0]), xycoords='data',
                    xytext=(0,25), textcoords='offset points',
                    ha='right', va='bottom')
    #ax.plot(0.3,-1.0, marker='s', linestyle='None', color='magenta')
    ax.set_xlim(-0.9, 1.75)
    ax.set_ylim(-3.8, -0.4)
    ax.annotate(r'{0:.0f} Myr'.format(age/1.e6), size=24,
                xy=(0.98, 0.02), xycoords='axes fraction', ha='right', va='bottom')
    ax.errorbar(np.log10(0.303), np.log10(0.10), yerr=(0.15), color='k',
                marker='s', markeredgecolor='None', markersize=6)
    ax.errorbar(np.log10(0.303), np.log10(0.078), yerr=(0.2), color='orange',
                marker='p', markeredgecolor='None', markersize=9)
    ax.errorbar(np.log10(0.303), np.log10(0.017), yerr=(0.3), color='darkred',
                marker='p', markeredgecolor='None', markersize=9)

fig.subplots_adjust(hspace=0, wspace=0, right=0.98, top=0.98, left=0.08, bottom=0.1)

[fig.axes[i].set_xlabel(r'log [O III]/[O II]', size=18) for i in[3,4,5]]
[fig.axes[i].set_ylabel(r'log He$\,$II$\lambda$1640/H$\beta$', size=18)
 for i in [0,3]]
plt.setp([a.get_xticklabels() for a in fig.axes[0:3]], visible=False);
plt.setp([fig.axes[i].get_yticklabels() for i in [1,2,4,5]], visible=False);

fig.savefig('./f24.pdf', dpi=300)

# Figure 25

In [ ]:
sxname = 'log_SII_Ha' #SII Ha
syname = 'log_OIII_Hb' #Oiiiha
ylim= [-2.3, 1.3]
xlim=[-2.1, 0.1]
ratio='SII'
add_starlab=True
age = 1.0e6
xlab, ylab = r'log [S II]$\lambda$6716,6731 / H$\alpha$', r'log [O III]$\lambda$5007 / H$\beta$'
xname, yname = sxname, syname

fig, ax = plt.subplots(1, figsize=(6.5,5.5))
plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax, cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
vzBPT(True, ax=ax, line_ratio=ratio)
SII = np.linspace(-3.0, 0.31)
OIII = 0.72/(SII - 0.32) + 1.3
ax.plot(SII, OIII, color='k', lw=2)
altBPT(mau, age=age, ax=ax, xname=xname, yname=yname, add_starlab=add_starlab,
       yclims=(-2.0, 0.5),
       xlim=xlim, ylim=ylim)

lab = "{0:.0f} Myr (MIST)".format(age/1.e6)
ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)
ax.set_xlabel(xlab, size=16)
ax.set_ylabel(ylab, size=16)
fig.subplots_adjust(top=0.98, right=0.98, left=0.12, bottom=0.12, wspace=0, hspace=0)
fig.savefig('./f25a.pdf', dpi=300)


In [ ]:
sxname = 'NII/OII'
syname = 'OIII/OII'
ylim= [-2.3, 1.9]
xlim=[-1.6, 0.8]
ratio='OII'
age = 1.0e6
xlab, ylab = r'log [N II]/[O II]', r'log [O III]/[O II]'
xname, yname = 'log_NII_OII', 'log_OIII_OII'

ages = [1.0e6, 5.0e6]
fig, ax = plt.subplots(1, figsize=(6.5,5.5))
add_starlab=True
plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax, 
            cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
vzBPT(True, ax=ax, line_ratio=ratio)
altBPT(mau, age=age, ax=ax, xname=xname, yname=yname,
       yclims=(-2.0, 0.5),lw=3,
       xlim=xlim, ylim=ylim, add_starlab=add_starlab)
lab = "{0:.0f} Myr (MIST)".format(age/1.e6)
ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)
ax.set_xlabel(xlab, size=16)
ax.set_ylabel(ylab, size=16)
fig.subplots_adjust(top=0.98, right=0.98, left=0.12, bottom=0.12, wspace=0, hspace=0)
fig.savefig('./f25b.pdf', dpi=300)


# Figure 26

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/output_pAGB/'
mod_prefix='PAU'
# Read in output
pau = ob.allmods(dir_, mod_prefix)
[mod.__setattr__('pAGB', mod.fbhb) for mod in pau.mods]
pau.add_arrs('pAGB')
pau.pAGB_vals = np.unique(pau.pAGB)

for mod in pau.mods:
    qs = np.array([mob.logQ for mob in pau.mods if mob.pAGB == mod.pAGB])
    q0 = np.min(qs)
    mod.Q_orig = q0
    mod.logM = mod.logQ - q0
pau.add_arrs('logM')
pau.logM_vals = np.unique(pau.logM)

In [ ]:
sxname = 'logNHa'
syname = 'log_OIII_Hb'
sxlim=[-1.7, 0.0]
sylim=[-2.4, 1.2]
age=2.82e9
bpt_inds = ['log_NIIb_Ha', 'log_OIIIb_Hb']
fig, ax = plt.subplots(1, figsize=(6.5, 5.5))

plot_2Dhist(sample[sxname], sample[syname], ranges=[sxlim, sylim], ax=ax,
            cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
#makeBPT(pau, ax=ax)
NII = np.linspace(-4.5, 0.04)
OIII = 0.61/(NII - 0.05) + 1.3
ax.plot(NII, OIII, '--', color='k', lw=2)

NII = np.linspace(-4.5, 0.46, 100)
OIII = 0.61/(NII - 0.47) + 1.19
ax.plot(NII, OIII, color='k', lw=2)
nH_handles, nH_labels = [], []
R_handles, R_labels = [], []

ponce=0
ax.set_xlim(-1.4, 0.4)
ax.set_ylim(-2.3, 1.4)
nHcols = nColors(3, cname='Blues', minv=0.3, maxv=1.0)
Rcols = nColors(3, cname='Reds', minv=0.3, maxv=1.0)
for nH,c in zip([10., 100., 1000.], nHcols):
    for logR, color in zip([18., 19., 20.], Rcols):
        inds, = np.where((pau.nH == nH) & (pau.logR == logR) & (pau.logM > 4.) & (pau.pAGB == 1.0))
        sM = getColors(np.linspace(42., 49.))
        xarr = np.array([pau.log_NIIb_Ha[i] for i in inds])
        yarr = np.array([pau.log_OIIIb_Hb[i] for i in inds])
        lab = '__nolegend__'
        if logR == 20.:
            lab = r'${0:.0f}$'.format(np.log10(nH))
        handles, = ax.plot(xarr, yarr, 'o', markersize=8, color=c, markeredgecolor='None', label=lab, zorder=15)
        if lab != '__nolegend__':
            nH_handles.append(handles)
            nH_labels.append(lab)
        lab = '__nolegend__'
        if nH == 10.:
            lab = r'${0:.0f}$'.format(logR)
        handles, = ax.plot(xarr, yarr, color=color, lw=3, alpha=0.8, label=lab)
        if lab != '__nolegend__':
            R_handles.append(handles)
            R_labels.append(lab)
        if (nH == 1000.):
            for i in inds:
                lab = r'{0:.0f}'.format(pau.logU[i])
                x = pau.log_NIIb_Ha[i]
                y = pau.log_OIIIb_Hb[i]
                if float(lab) == -4.:
                    ponce += 1
                    if ponce <= 1:
                        ax.annotate(lab,
                                    xy=(x, y), size=16,
                                    xycoords='data',
                                    xytext=(x+0.05, y+0.1),
                                    textcoords='data',
                                    horizontalalignment='left',
                                    verticalalignment='top')
                else:
                    ax.annotate(lab,
                                xy=(x, y), size=16,
                                xycoords='data',
                                xytext=(x+0.05, y+0.1),
                                textcoords='data',
                                horizontalalignment='left',
                                verticalalignment='top')
xlabel=r'$\log$ [N II]$\;\lambda$6584 / H$\alpha$'
ylabel=r'$\log$ [O III]$\;\lambda$5007 / H$\beta$'
#zau.makeBPT(data_only=True, ax=ax, plot_data=label_data, line_ratio=line_ratio)
ax.set_xlabel(xlabel, fontsize=16)
ax.set_ylabel(ylabel, fontsize=16)
plt.xlim(-2.25, 0.25)
plt.ylim(-1.75,1.45)

biglab = r'$\log \mathcal{{U}}_0~/\log \hat{{Q}}_{{\mathrm{{H}}}}$'
biglab = r'$\log \mathcal{{U}}_0$'
ax.annotate(biglab, xy=(0.45, 0.92), xycoords='axes fraction', size=16)

nH_leg = ax.legend(nH_handles, nH_labels, loc=6, frameon=False, numpoints=1,
                   fontsize=16, title=r'log n$_{\mathrm{H}}$')
ax.add_artist(nH_leg)
plt.setp(nH_leg.get_title(), fontsize=16)
R_leg = ax.legend(R_handles, R_labels, loc=3, frameon=False,
                  fontsize=16, title=r'log R$_{\mathrm{inner}}$')
ax.add_artist(R_leg)
plt.setp(R_leg.get_title(), fontsize=16)
fig.subplots_adjust(bottom=0.12, top=0.98, left=0.12, right=0.98)
fig.savefig('./f26.pdf', dpi=300)

# Figure 27

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputZAUpAGB/'
mod_prefix='PAU'
# Read in output
pau = ob.allmods(dir_, mod_prefix)
[mod.__setattr__('pAGB', mod.fbhb) for mod in pau.mods]
pau.add_arrs('pAGB')
pau.pAGB_vals = np.unique(pau.pAGB)
print pau.pAGB_vals

In [ ]:
age=2.82e9
bpt_inds = ['log_NIIb_Ha', 'log_OIIIb_Hb']
fig, ax = plt.subplots(1, figsize=(6.5,5.5))
plot_2Dhist(sample[sxname], sample[syname], ranges=[sxlim, sylim], ax=ax,
            cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
#makeBPT(pau, ax=ax, line_ratio='NIIb')
NII = np.linspace(-4.5, 0.04)
OIII = 0.61/(NII - 0.05) + 1.3
ax.plot(NII, OIII, '--', color='k', lw=2)

NII = np.linspace(-4.5, 0.46, 100)
OIII = 0.61/(NII - 0.47) + 1.19
ax.plot(NII, OIII, color='k', lw=2)
###
x_name = 'logU'
y_name = 'pAGB'
grid_x = pau.__getattribute__(x_name+'_vals')
grid_y = pau.__getattribute__(y_name+'_vals')

sMp = getColors(np.log10(grid_y), cname='inferno_r', maxv=1.0, minv=0.2)
sMu = getColors(grid_x, cname='Blues', maxv=1.0, minv=0.2)

for uu in grid_x:
    use_mods = [mod for mod in pau.mods if ((mod.age == age)
                                            & (mod.__getattribute__(x_name) == uu))]
    x = np.array([mod.__getattribute__(bpt_inds[0]) for mod in use_mods])
    y = np.array([mod.__getattribute__(bpt_inds[1]) for mod in use_mods])
    ax.plot(x, y, lw=4, color=sMu.to_rgba(uu), markeredgecolor='None')
    if uu == grid_x[0]:
        lab = r"$\log\,\mathcal{{U}}_0 = {0:.1f}$".format(uu)
        horizontalalignment='right'
    else:
        lab = r"${0:.1f}$".format(uu)
        horizontalalignment='right'
    ax.annotate(lab, size=16,
                xy=(x[0], y[0]),
                xycoords='data',
                xytext=(x[0], y[0]),
                textcoords='data',
                horizontalalignment=horizontalalignment, verticalalignment='top')

for vv in grid_y:
    use_mods = [mod for mod in pau.mods if ((mod.age == age)
                                            & (mod.__getattribute__(y_name) == vv))]
    x = np.array([mod.__getattribute__(bpt_inds[0]) for mod in use_mods])
    y = np.array([mod.__getattribute__(bpt_inds[1]) for mod in use_mods])
    ax.plot(x, y, lw=4, color=sMp.to_rgba(np.log10(vv)), markeredgecolor='None')
    if vv == grid_y[-1]:
        lab = r"pAGB$ = 10^{{{0:.0f}}}$".format(np.log10(vv))
        offset = 0.15
        horizontalalignment='left'
    elif vv == grid_y[-2]:
        lab = ''
    else:
        lab = r"$10^{{{0:.0f}}}$".format(np.log10(vv))
        offset = 0.0
        horizontalalignment='right'
    ax.annotate(lab, size=16,
                xy=(x[-1], y[-1]),
                xycoords='data',
                xytext=(x[-1], y[-1]+offset),
                textcoords='data', horizontalalignment=horizontalalignment)
plt.xlim(-2.25, 0.25)
plt.ylim(-1.4,1.45)
xlabel=r'$\log$ [N II]$\;\lambda$6584 / H$\alpha$'
ylabel=r'$\log$ [O III]$\;\lambda$5007 / H$\beta$'
ax.set_xlabel(xlabel, fontsize=16)
ax.set_ylabel(ylabel, fontsize=16)
fig.subplots_adjust(left=0.12, right=0.98, top=0.98, bottom=0.12)
fig.savefig('./f27b.pdf', dpi=300)
#fig.savefig('./hotstars/pAGB_BPT.eps', format='eps', dpi=1000)

# Figure 28

In [ ]:
xlab, ylab = r'log [S II]$\lambda$6716,6731 / H$\alpha$', r'log [O III]$\lambda$5007 / H$\beta$'
xratio = 'log_SII_Ha'
yratio = 'log_OIIIb_Hb'

sxname = 'log_SII_Ha' #SII Ha
syname = 'log_OIII_Hb' #Oiiiha
ylim= [-1.6, 1.6]
xlim=[-2.1, 0.3]
ratio='SII'


fig,ax = plt.subplots(1,figsize=(6.5,5.5))
plot_2Dhist(sample[sxname], sample[syname], ranges=[xlim, ylim], ax=ax, cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
altBPT(pau, ax=ax, bpt_inds=[xratio, yratio], line_ratio=ratio, do_label=True)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xlabel(xlab, size=16)
ax.set_ylabel(ylab, size=16)
fig.subplots_adjust(left=0.12, right=0.98, top=0.98, bottom=0.12)
SII = np.linspace(-3.0, 0.31)
OIII = 0.72/(SII - 0.32) + 1.3
ax.plot(SII, OIII, color='k', lw=2)
fig.savefig('./f28.pdf'.format(ratio), dpi=300)

# Figure 29

In [ ]:
dir_ = '/astro/users/ebyler/research/newem/outputCKC/'
mod_prefix='ZAU'
zau = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)
dir_ = '/astro/users/ebyler/research/newem/outputDCKC/'
mod_prefix='ZAU'
dau = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
[zau.mods[i].__setattr__('TempDiff', ((zau.mods[i].T0-dau.mods[i].T0)/zau.mods[i].T0)) for i in range(len(zau.mods))]


key = 'TempDiff'
lab = r'[ $\mathrm{T}_e$ - $\mathrm{T}_e$(dust) ] / $\mathrm{T}_e$'
cname = 'gist_heat'
ylabel=r'log Z/Z$_{\odot}$'
xlabel=r'log $\mathcal{U}_0$'
fig, ax = plt.subplots(1, figsize=(5,5))
zau.pxl_plot(zval=key, clab=lab, ax=ax,
             const='age', cval=1.0e6, cname=cname,
             yval='logZ', ylab=r'$log Z/Z$_{\odot}$',
             xval='logU', xlab=r'$\log \mathcal{U}_0$', calc_aspect=False)
ax.set_ylabel(ylabel, size=16)
ax.set_xlabel(xlabel, size=16)
fig.subplots_adjust(left=0.13, bottom=0.125, top=0.98, right=0.925)
fig.savefig('./f29.pdf', dpi=300)

# Figure 30

In [ ]:
sxname = 'logNHa'
syname = 'log_OIII_Hb'
xlim=[-1.7, 0.1]
ylim=[-2.4, 1.2]

ratio='NIIb'
age = 1.0e6
xlab=r'$\log$ [N II]$\;\lambda$6584 / H$\alpha$'
ylab=r'$\log$ [O III]$\;\lambda$5007 / H$\beta$'
xname, yname = 'log_NIIb_Ha', 'log_OIIIb_Hb'

fig, ax = plt.subplots(1, figsize=(6.5,5.5))
plot_2Dhist(sample[sxname], sample[syname], ranges=[[-2.0,0.0], ylim], ax=ax, cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
#vzBPT(True, ax=ax, line_ratio=ratio)
NII = np.linspace(-4.5, 0.04)
OIII = 0.61/(NII - 0.05) + 1.3
ax.plot(NII, OIII, '--', color='k', lw=2)

NII = np.linspace(-4.5, 0.46, 100)
OIII = 0.61/(NII - 0.47) + 1.19
ax.plot(NII, OIII, color='k', lw=2)
altBPT(dau, age=age, ax=ax, xname=xname, yname=yname, alpha=0.7,add_star=False,
       yclims=(-1.0, 0.3), ycname='Reds', xcname='Reds', add_starlab=add_starlab,
       xlim=xlim, ylim=ylim,xv=(0.8, 0.8), yv=(0.8, 0.8), lw=2)
altBPT(zau, age=age, ax=ax, xname=xname, yname=yname, alpha=0.7,add_star=False,
   yclims=(-1.0, 0.3), ycname='Blues', xcname='Blues', add_starlab=add_starlab,
   xlim=xlim, ylim=ylim,xv=(0.8, 0.8), yv=(0.8, 0.8), lw=2)

#lab = "{0:.0f} Myr".format(age/1.e6)
#ax.annotate(lab, xy=(0.98, 0.98), xycoords='axes fraction', ha='right', va='top', size=20)
ax.set_xlabel(xlab, size=16)
ax.set_ylabel(ylab, size=16)

colors = nColors(2, cname='RdYlBu', minv=0.1, maxv=0.9)
ax.annotate(r'Grains', xy=(0.4, 0.82), xycoords='axes fraction', fontsize=22,
                xytext=(0.6,0.9), textcoords='axes fraction',
                 arrowprops=dict(arrowstyle='wedge', connectionstyle='arc3, rad=0.3', fc=colors[0], ec='None'))
ax.annotate(r'No Grains', xy=(0.4, 0.76), xycoords='axes fraction', fontsize=22,
                xytext=(0.7,0.82), textcoords='axes fraction',
                 arrowprops=dict(arrowstyle='wedge', connectionstyle='arc3, rad=0.15', fc=colors[1], ec='None'))

fig.subplots_adjust(top=0.98, right=0.98, left=0.12, bottom=0.12, wspace=0, hspace=0)
fig.savefig('./f30.pdf', dpi=300)


# Figure 31

In [ ]:
dir_='/astro/users/ebyler/research/newem/output_sb/'
mod_prefix = 'ZAU'
sb = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
dir_='/astro/users/ebyler/research/newem/outputCKC/'
mod_prefix = 'ZAU'
za = ob.allmods(dir_, mod_prefix, read_out=True, read_rad=True)

In [ ]:
sxname = 'logNHa'
syname = 'log_OIII_Hb'
xlim=[-1.7, 0.1]
ylim=[-2.4, 1.2]


ages = [za.age_vals[1], za.age_vals[2], za.age_vals[4]]
nages = len(ages)
colors = nColors(nages, cname='inferno', maxv=0.7, minv=0.2)
fig, axes = plt.subplots(1,3, figsize=(9,3.5))
biglabs = [r'1 Myr',r'2 Myr',r'3 Myr']
biglabs = [r'{0:.0f} Myr'.format(age/1.e6) for age in ages]
xlab=r'$\log$ [N II]$\;\lambda$6584 / H$\alpha$'
ylab=r'$\log$ [O III]$\;\lambda$5007 / H$\beta$'
colors = nColors(2, cname='inferno', minv=0.3, maxv=0.6)
for age, ax, biglab in zip(ages, fig.axes, biglabs):
    pd=False
    var_label=False
    plot_2Dhist(sample[sxname], sample[syname], ranges=[[-2.0,0.0], ylim], ax=ax,
            cmap=plt.cm.binary, vmax=1.e4, plot_cb=False)
    NII = np.linspace(-4.5, 0.04)
    OIII = 0.61/(NII - 0.05) + 1.3
    ax.plot(NII, OIII, '--', color='k', lw=2)
    NII = np.linspace(-4.5, 0.46, 100)
    OIII = 0.61/(NII - 0.47) + 1.19
    ax.plot(NII, OIII, color='k', lw=2)
    vzBPT(True, ax=ax, line_ratio='NIIb')
    za.makeBPT(ax=ax, color=colors[0], line_ratio='NIIb',
                par_label=r'Padova+Geneva',
                plot_data=pd, var_label=False,
                const1='age', val1=age,
                const2='logR', val2=19.0,
                cut_z=(-1.0,0.3),
                const3='nH', val3=100.0, lw=3)
    sb.makeBPT(ax=ax, color=colors[1], line_ratio='NIIb', alpha=0.75,
               par_label=r'SB-99',
               plot_data=False, var_label=False,
               const1='age', val1=age,
               const2='logR', val2=19.0,
               cut_z=(-1.0,0.3),
               const3='nH', val3=100.0, lw=3)
    ax.set_xlim(-3.25, 0.25)
    ax.set_ylim(-2.5, 1.75)
    ax.set_xlabel(' ')
    ax.set_ylabel(' ')
    ax.annotate(biglab, xy=(0.05, 0.88), xycoords='axes fraction', size=22)
ax.legend_.remove()
[ax.set_xlabel(xlab, size=16) for ax in fig.axes]
fig.axes[0].set_ylabel(ylab, size=16)
#plt.setp([a.get_xticklabels() for a in fig.axes], visible=False);
plt.setp([a.get_yticklabels() for a in fig.axes[1::]], visible=False);
fig.subplots_adjust(hspace=0, wspace=0)
fig.subplots_adjust(left=0.08, right=0.98, top=0.98, bottom=0.15)
fig.axes[0].annotate(r'This work', xy=(0.4, 0.78), xycoords='axes fraction', fontsize=16,
                xytext=(0.6,0.9), textcoords='axes fraction',
                 arrowprops=dict(arrowstyle='wedge', connectionstyle='arc3, rad=0.3', fc=colors[0], ec='None'))
fig.axes[0].annotate(r'SB-99', xy=(0.4, 0.78), xycoords='axes fraction', fontsize=16,
                xytext=(0.7,0.82), textcoords='axes fraction',
                 arrowprops=dict(arrowstyle='wedge', connectionstyle='arc3, rad=0.1', fc=colors[1], ec='None'))
fig.savefig('./f31.pdf', dpi=300)